In [1]:
!pip install Flask pyngrok line-bot-sdk requests openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.4/758.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_key = userdata.get('ngrok_key')
line_access_token = userdata.get('line_access_token')
line_secret = userdata.get('line_secret')
port = 5000

In [3]:
import os
from pyngrok import ngrok

In [5]:
ngrok.set_auth_token(ngrok_key)
tunnel = ngrok.connect(5000, name="linebot_tunnel")
print(tunnel.public_url)

https://bc1c-34-106-55-99.ngrok-free.app


In [6]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text),
                    TextMessage(text=event.message.text)]
            )
        )

@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='定位訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/May/2024 17:19:09] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}
BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"510025806354055313","quoteToken":"kGaKkeYBtLpZUtkVJqJ0Dp4rw6doBD2VvmKAN8lkx-cEdYpynpieEmko8lDbz7UQoPBQ597SQtMycHIRDCO-2Pr3FuwMBWqAEZcKxb1ZeQ3Up7xod7VqeM_v2ObLd2W_bNGnsw9Oz7fSI4oF16WkCw","text":"測試"},"webhookEventId":"01HYXH12R66JH2XDHZQ455KHJN","deliveryContext":{"isRedelivery":false},"timestamp":1716830374155,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"b845f075bf624c419bb9f6a387f2464a","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [27/May/2024 17:19:35] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"sticker","id":"510025818853081297","quoteToken":"6yfVUQ1gpm8nckrCoxRPaUTIOoNvWvrET09Jwk6UClxawE7J1MexnonR2XPe-qMQNIlKcOiSeelcjJHbDsaveSLRkIJ68sTKAMy-BZBKykL9nXy2b_xisPYSM94V9oZR6igu69FcLRt9k15eRQ2mdA","stickerId":"52002734","packageId":"11537","stickerResourceType":"ANIMATION","keywords":["line","Celebrate",":o","Yay","Congratulations","jolt","Surprised","ohyeah","astonish","brown","OMG","yeah","Shock","bewildered","amaze"]},"webhookEventId":"01HYXH1A0XA58DMMAX7PTCZP4D","deliveryContext":{"isRedelivery":false},"timestamp":1716830381604,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"2a5398fcb1754de9b96fa6877ff7e67f","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [27/May/2024 17:19:42] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"location","id":"510025847458234517","latitude":25.00295,"longitude":121.564798,"address":"116台灣台北市文山區萬寧街36巷11號"},"webhookEventId":"01HYXH1TMD7JC2A9RY37Q17PT7","deliveryContext":{"isRedelivery":false},"timestamp":1716830398651,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"5e1ca9f728054cdc82bcb66175bc1d79","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [27/May/2024 17:20:00] "POST / HTTP/1.1" 200 -
